# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import gkey
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
csv_path = "city_data.csv"
city_data_df = pd.read_csv(csv_path, encoding="UTF-8")
city_data_df.head(1)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#Create Location Variables
locations = city_data_df[["Latitude", "Longitude"]].astype(float)

# Create Humidity Variable
humidity_rate = city_data_df["Humidity"].astype(float)

In [ ]:
# Create a Humidity Heatmap layer
#adding latitude and longitude for each country
fig_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}

fig = gmaps.figure(map_type="TERRAIN")
fig = gmaps.figure(layout=fig_layout, center=(26.820553, 30.802498), zoom_level=1.75)
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=95,
                                 point_radius = 2,gradient=['rgba(0,0,0,0)','rgb(2, 199, 22)','yellow','red'])

fig.add_layer(humidity_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Create a new DataFrame that looks at non-humid areas
low_humid_df = city_data_df.loc[city_data_df["Humidity"] < 65]
low_humid_df.head(1)

In [ ]:
# Create a new DataFrame that looks at non arid areas
ideal_humid_df = low_humid_df.loc[low_humid_df["Humidity"] >50]
ideal_humid_df.head(1)

In [ ]:
ideal_climate_df = ideal_humid_df.loc[ideal_humid_df["Temperature"] <75]
ideal_climate_df['Hotel Name'] = ""
ideal_climate_df.head(5)

In [ ]:
#Redefine Location Variables to ideal dataframe
locations = ideal_climate_df[["Latitude", "Longitude"]].astype(float)

In [ ]:
# Create dictionary of parameters that will update throuh each iteration
for city, row in ideal_climate_df.iterrows():
    base_hotel_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {'location': locations, 'rankby': 'distance', 'type': 'hotel', 'api': gkey}

    hotel_df = requests.get(base_hotel_url, params=params)

In [ ]:
print(f"url = {base_hotel_url}{params}")

In [ ]:
hotel_df

In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'location': coordinates,
    'type': 'hotel',
    'rankby': 'distance',
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for cityIndex_num, row in ideal_climate_df.iterrows():     ## for item_variable, row_varible, "in" dataframe_varible.iterrows():

    # get restaurant type from df
    coordinates = row['Latitude'], row['Longitude']

    # add keyword to params dict
    params['location'] = coordinates

    # assemble url and make API request
    print(f"Retrieving Results for Index {cityIndex_num}: {coordinates}.")
    response = requests.get(base_url, params=params)
    
    # extract results
    results = response.json()
    print(results)

In [ ]:
print(f"Search url = {base_hotel_url}{params}")

In [ ]:
ideal_climate_df.append['Hotel Name']

In [ ]:
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Name": ideal_climate_df['Hotel Name'], "City": ideal_climate_df['City'], \
                      "Country": ideal_climate_df['Country']])
fig.add_layer(markers)

In [ ]:
# Save reduced to ideal parameter data to csv

ideal_climate_df.to_csv("hotel_df.csv", encoding="utf-8", index=False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
